In [1]:
library(text2vec)
library(glmnet)
library(slam)
library(pROC)

Loading required package: Matrix

Loaded glmnet 4.1-4

Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var




### Split generation

In [1]:
data <- read.table("alldata.tsv", stringsAsFactors = FALSE,
                  header = TRUE)
testIDs <- read.csv("project3_splits.csv", header = TRUE)
for(j in 1:5){
  dir.create(paste("split_", j, sep=""))
  train <- data[-testIDs[,j], c("id", "sentiment", "review") ]
  test <- data[testIDs[,j], c("id", "review")]
  test.y <- data[testIDs[,j], c("id", "sentiment", "score")]
  
  tmp_file_name <- paste("split_", j, "/", "train.tsv", sep="")
  write.table(train, file=tmp_file_name, 
              quote=TRUE, 
              row.names = FALSE,
              sep='\t')
  tmp_file_name <- paste("split_", j, "/", "test.tsv", sep="")
  write.table(test, file=tmp_file_name, 
              quote=TRUE, 
              row.names = FALSE,
              sep='\t')
  tmp_file_name <- paste("split_", j, "/", "test_y.tsv", sep="")
  write.table(test.y, file=tmp_file_name, 
            quote=TRUE, 
            row.names = FALSE,
            sep='\t')
}

### Generate Initial document term matrix

In [2]:
# load all data and clean the html tags.
train = read.table("alldata.tsv",
                   stringsAsFactors = FALSE,
                   header = TRUE)
train$review = gsub('<.*?>', ' ', train$review)

# construct DT (DocumentTerm) matrix (maximum 4-grams).
stop_words = c("i", "me", "my", "myself", 
               "we", "our", "ours", "ourselves", 
               "you", "your", "yours", 
               "their", "they", "his", "her", 
               "she", "he", "a", "an", "and",
               "is", "was", "are", "were", 
               "him", "himself", "has", "have", 
               "it", "its", "the", "us")
it_train = itoken(train$review,
                  preprocessor = tolower, 
                  tokenizer = word_tokenizer)
tmp.vocab = create_vocabulary(it_train, 
                              stopwords = stop_words, 
                              ngram = c(1L,4L))
tmp.vocab = prune_vocabulary(tmp.vocab, term_count_min = 10,
                             doc_proportion_max = 0.5,
                             doc_proportion_min = 0.001)
dtm_train  = create_dtm(it_train, vocab_vectorizer(tmp.vocab))

In [4]:
# trim the vocabulary size to 2K using t-test
v.size = dim(dtm_train)[2]
ytrain = train$sentiment

summ = matrix(0, nrow=v.size, ncol=4)
summ[,1] = colapply_simple_triplet_matrix(
  as.simple_triplet_matrix(dtm_train[ytrain==1, ]), mean)
summ[,2] = colapply_simple_triplet_matrix(
  as.simple_triplet_matrix(dtm_train[ytrain==1, ]), var)
summ[,3] = colapply_simple_triplet_matrix(
  as.simple_triplet_matrix(dtm_train[ytrain==0, ]), mean)
summ[,4] = colapply_simple_triplet_matrix(
  as.simple_triplet_matrix(dtm_train[ytrain==0, ]), var)

n1 = sum(ytrain); 
n = length(ytrain)
n0 = n - n1

myp = (summ[,1] - summ[,3])/
  sqrt(summ[,2]/n1 + summ[,4]/n0)

words = colnames(dtm_train)
id = order(abs(myp), decreasing=TRUE)[1:2000]
pos.list = words[id[myp[id]>0]]
neg.list = words[id[myp[id]<0]]

In [34]:
words[id]

ERROR: Error in eval(expr, envir, enclos): object 'words' not found


In [6]:
vectorizer = vocab_vectorizer(create_vocabulary(words, 
                                                ngram = c(1L, 2L)))
dtm_reduced = create_dtm(it_train, vectorizer)

In [7]:
# use Lasso (with logistic regression) to trim the vocabulary size to 2K.
set.seed(7568)
tmpfit = glmnet(x = dtm_reduced, 
                y = train$sentiment, 
                alpha = 0.05,
                family='binomial')
tmpfit$df

[1]     0     1     2     3     4     5     7    14    18    26    38    49
 [13]    60    75   100   126   152   177   208   242   288   330   371   419
 [25]   470   528   584   674   754   855   935  1052  1187  1321  1472  1645
 [37]  1838  2076  2334  2612  2891  3238  3583  3951  4336  4779  5259  5727
 [49]  6186  6643  7099  7612  8112  8611  9082  9564 10027 10477 10908 11343
 [61] 11763 12196 12600 12945 13316 13662 14041 14359 14686 14984 15262 15532
 [73] 15780 16032 16252 16466 16686 16887 17057 17222 17396 17546 17683 17832
 [85] 17978 18111 18235 18352 18506 18628 18726 18842 18942 19052 19158 19256
 [97] 19347 19434 19519 19601

In [17]:
tmpfit$df[31]

[1] 935

In [18]:
# pick the largest df among the beta values thar are less than 1K 
#(which turns out to be the 31st column), and store the corresponding words in myvocab
myvocab = colnames(dtm_reduced)[which(tmpfit$beta[, 31] != 0)]

In [19]:
vectorizer = vocab_vectorizer(create_vocabulary(myvocab, 
                                                  ngram = c(1L, 2L)))
dtm_test = create_dtm(it_train, vectorizer)

In [21]:
testfit = glmnet(x = dtm_test, 
                 y = train$sentiment, 
                 alpha = 0.05,
                 family='binomial')

In [22]:
pred = predict(testfit, newx=dtm_test, s=0.01)

In [23]:
roc_obj <- roc(train$sentiment, c(pred))
pROC::auc(roc_obj)

Setting levels: control = 0, case = 1

Setting direction: controls < cases



Area under the curve: 0.9715

In [27]:
# Write vocab to text file
some.strs <- c(myvocab)
write.table(some.strs, file = "myvocab.txt",
            quote = FALSE,
            row.names = FALSE,
            col.names = FALSE,
            sep = "\n")

### Submitted code

In [2]:
# This cell is appended to the top of the submitted code
library(text2vec)
library(glmnet)
myvocab <- scan(file = "myvocab.txt", what = character())

In [32]:
# this loop is for our purposes, we will not submit it
scores = {}
for (j in 1:5) {
    setwd(paste("split_", j, sep=""))
    # below this comment is submitted code unless otherwise stated
    train = read.table("train.tsv",
                       stringsAsFactors = FALSE,
                       header = TRUE)
    train$review <- gsub('<.*?>', ' ', train$review)

    # Create matrix corresponding to vocab
    it_train = itoken(train$review,
                        preprocessor = tolower, 
                        tokenizer = word_tokenizer)
    vectorizer = vocab_vectorizer(create_vocabulary(myvocab, 
                                                      ngram = c(1L, 2L)))
    dtm_train = create_dtm(it_train, vectorizer)
    set.seed(7568)
    fit = glmnet(x = dtm_train, 
                    y = train$sentiment, 
                    alpha = 0.05,
                    family='binomial')
    
    #####################################
    # Load test data, and 
    # Compute prediction
    #####################################
    test <- read.table("test.tsv", stringsAsFactors = FALSE,
                    header = TRUE)

    test$review <- gsub('<.*?>', ' ', test$review)

    # Create matrix corresponding to vocab
    it_test = itoken(test$review,
                        preprocessor = tolower, 
                        tokenizer = word_tokenizer)
    vectorizer = vocab_vectorizer(create_vocabulary(myvocab, 
                                                ngram = c(1L, 2L)))
    dtm_test = create_dtm(it_test, vectorizer)
    
    prob = predict(fit, newx=dtm_test, s=0.05)
    output = data.frame(id=c(test$id),
                        prob=c(prob))
    #####################################
    # Store your prediction for test data in a data frame
    # "output": col 1 is test$id
    #           col 2 is the predicted probs
    #####################################
    write.table(output, file = "mysubmission.txt", 
                row.names = FALSE, sep='\t')
    
    # Below this is not submitted code
    # move "test_y.tsv" to this directory
    test.y <- read.table("test_y.tsv", header = TRUE)
    pred <- read.table("mysubmission.txt", header = TRUE)
    pred <- merge(pred, test.y, by="id")
    roc_obj <- roc(pred$sentiment, pred$prob)
    score = pROC::auc(roc_obj)
    scores[j] = score
    setwd('../')
    }

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases



In [33]:
scores

[1] 0.9640572 0.9635631 0.9634647 0.9640653 0.9633994